In [0]:
!pip install -q six numpy scipy matplotlib scikit-image opencv-python imageio
!pip install -q keras imgaug
!pip install -q tensorboardcolab
!pip install -q git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
import keras
from keras_contrib.callbacks import CyclicLR, DeadReluDetector, SnapshotCallbackBuilder, SnapshotModelCheckpoint
import imgaug as ia
from imgaug import augmenters as iaa
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from keras_contrib.applications import DenseNet

Using TensorFlow backend.


In [0]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,SeparableConv2D
from keras.layers import Concatenate
from keras.optimizers import SGD,Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
# from keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,LambdaCallback
import os,sys,math
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
import google
colab_dir='./'
file_name='EIP_CIFAR_10'
if hasattr(google,'colab'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    colab_dir='/content/gdrive/My Drive/Colab Notebooks/'
model_file=colab_dir+file_name+'.h5'
weights_dir=colab_dir+'weights/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# !git clone https://github.com/pjreddie/darknet.git dkn
# !mv -v dkn/* ./
# !sed -i  's/GPU=0/GPU=1/g' Makefile
# !make -j4

In [0]:
# !wget https://www.hublot.com/images/News_2018/match_for_solidarity_gallery_13.jpg -O input_image.jpg
# !wget https://pjreddie.com/media/files/yolov2-tiny.weights -O y.weights
# !wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov2-tiny.cfg -O y.cfg

In [0]:
# !./darknet detect y.cfg y.weights input_image.jpg

In [0]:
#Augmentation and resizing
#augment and then concat samples with original
def augment(dataset,flip=0.5,blur=1.0):
    ia.seed(1)
    seq = iaa.Sequential([iaa.Crop(px=(0, 16)),iaa.Fliplr(0.5),iaa.GaussianBlur(sigma=(0, 3.0))], random_order=True)   
    return seq.augment_images(dataset)

def augmenter(X,y,start=0,end=1,append=False):
    ln=len(X)
    print('Before augmentation:',X.shape,y.shape)
    start=int(start*ln)
    end=int(end*ln)
    if append:
      new_X=augment(X)[start:end]
      new_y=y[start:end]
      X=np.concatenate((X,new_X))
      y=np.concatenate((y,new_y))
    else:
      X=augment(X)
    print('After augmentation:',X.shape,y.shape)
    return (X,y)

#26x26 is almost half of 32x32. 22x22 maybe too small even though its exact half.
def resize_imgs(imgs,shape=(26,26)):
    seq = iaa.Sequential([iaa.Scale({"height": shape[0], "width": shape[1]})])
    return seq.augment_images(imgs)

In [0]:
modval=50
max_lr=0.1

def cosine_anneal_schedule(t,lr):
  global modval,max_lr
  modval=int(modval)
  cos_inner = (np.pi * (t % modval))/ modval
  cos_out = np.cos(cos_inner) + 1
  return float(max_lr / 2 * cos_out)

def load_prev_model(model_small=None):
#     print(os.popen('du -sh '+colab_dir+'*').read())
    last_best=os.popen('du -sh '+colab_dir+r'*|sed -r "s/^[0-9\.]+[MK]?\s(.*)$/\1/g;"|tail -1').read().strip()
    model_prev=None
    try:
        print('Attempting to load last best model from file - ',end='')
        last_best_fn=last_best.split('/')[-1]
        last_best_epoch=last_best_fn.split('.')[1].split('-')[0]
        model_prev=load_model(last_best)
        print('Sucess\nLoaded '+last_best_fn, 'epoch :', last_best_epoch)
    except Exception as e:
        print('Failed!\n',e)
        try:
            print('Attempting to load last saved model from file - ',end='')
            model_prev = load_model(model_file)
            print('Sucess\nLoaded model from file',model_file)
        except Exception as e:
    #         print(str(e), 'at line ', sys.exc_info()[2].tb_lineno)
            print('Failed!\n',e)
            try:
                print('Attempting to load in memory, small model - ',end='')
                if len(model_small.layers)>1:
                    model_prev=model_small
                print('Sucess')
            except Exception as e:
                print('Failed!\n',e)
    return model_prev
  
def copy_weights(model_to,model_from):
    #model_to.set_weights(model_from.get_weights())
    s,err=0,0
    print('Trying to copy weights')
    try:
        if len(model_to.layers) >1 and len(model_from.layers) >1:
            pass
    except Exception as e:
        print('Inavlid models',model_to,model_from)
        return
    for new_layer, layer in zip(model_to.layers[1:], model_from.layers[1:]):
        s+=1
        try:
            new_layer.set_weights(layer.get_weights())
        except Exception as e:
            pass
    print('Done: errors:',err)
      
#new model with larger i/p layer
def larger_model(src_model,shape=(32,32,3)):
    new_input=Input(shape)
    src_model.layers.pop(0)
    new_output=src_model(new_input)
    new_model=Model(new_input,new_output)
    return new_model

# Load CIFAR10 Data
def load_data(resize=False,append=False,shape=(26,26),train_augment=True,test_augment=False):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    if resize:
        x_train=resize_imgs(x_train,shape)
        x_test=resize_imgs(x_test,shape)
    if train_augment:
      (x_train, y_train) = augmenter(x_train, y_train,end=1,append=append)
    if test_augment:
        (x_test, y_test) = augmenter(x_test, y_test,end=1,append=append)
    return (x_train, y_train,x_test, y_test)
#create a dnn model
def create_model(input_shape,num_layers,input_conv_filters=12,input=None):
    print('Creating model with input shape',input_shape)
    if input is None:
        input = Input(input_shape)
    First_Conv2D = Conv2D(input_conv_filters, (3,3), use_bias=False ,padding='same')(input)
    hidden_dense_blocks = dense_units_chain(n_dense_blocks,First_Conv2D,num_filter,dropout_rate,num_layers)
    Last_Block = add_denseblock(hidden_dense_blocks, num_filter, dropout_rate)
    output = output_layer(Last_Block)
    model = Model(inputs=[input], outputs=[output])
    return model

In [0]:
from keras.layers import GlobalAveragePooling2D
sc=False
# Dense Block
def add_denseblock(input,num_layers, growth_rate = 12, dropout_rate = 0.2,bn_layers=48,middle_layers=24):
    global compression
    temp = input
    for _ in range(num_layers):
        ######################################################################################################################################
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)        
        Conv2D_1_1_bottleneck = Conv2D(bn_layers, (1,1), use_bias=False ,padding='same',kernel_initializer='he_normal')(relu) #bottleneck  <===
        ###################################################################################################################################### |
        BatchNorm = BatchNormalization()(Conv2D_1_1_bottleneck) #extra layer                                                                 # |
        relu = Activation('relu')(BatchNorm) #extra layer                                                                                    # |
        Conv2D_1_1 = Conv2D(middle_layers, (1,1), use_bias=False ,padding='same',kernel_initializer='he_normal')(relu) #extra layer          # |
        ###################################################################################################################################### |
        BatchNorm = BatchNormalization()(Conv2D_1_1) #bottleneck   <================================================================
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(growth_rate, (3,3), use_bias=False ,padding='same',kernel_initializer='he_normal')(relu)
        ######################################################################################################################################|
        
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    GlobalAveragePooling = GlobalAveragePooling2D(data_format='channels_last')(relu)
#     flat = Flatten()(GlobalAveragePooling)
    output = Dense(num_classes, activation='softmax')(GlobalAveragePooling)
    
    return output

In [0]:
num_filter = 0 # add it up as you go
compression = 0.5
dropout_rate =0.2#0.2#.5
num_classes = 10

modval=20
max_lr=0.3

layer_models=[[96,48,24],[48,24,12]]
layer_models_index=1
bn_layers,middle_layers,growth_rate=layer_models[layer_models_index]
print('bn,middle,k :',layer_models[layer_models_index])
# num_layers=8 is kinda eq of standard Densenet-BC L=40 where 12*2 i.e 12*(BN_RELU_CONV*2), here its 8*3 i.e 8*(BN_RELU_CONV*3) 
num_layers=16#16#15
batch_size=64
initial_layers=12#12
hist=[]
keras_model=False
epochs=initial_epoch=0

bn,middle,k : [48, 24, 12]


In [0]:
model=None
if keras_model:
  model=DenseNet(input_shape=(32,32,3),depth=100,nb_dense_block=3,growth_rate=12,nb_filter=-1,bottleneck=True,reduction=0.5,dropout_rate=0.5,classes=10,pooling='avg')
else:
  input = Input(shape=(32, 32, 3,))
  First_Conv2D = Conv2D(initial_layers, (3,3), use_bias=False ,padding='same')(input)
  num_filter=initial_layers
  
  First_Block = add_denseblock(First_Conv2D,num_layers, growth_rate, dropout_rate,bn_layers,middle_layers)
  num_filter+=growth_rate*num_layers
  First_Transition = add_transition(First_Block, growth_rate*2, dropout_rate)
  num_filter=num_filter//2
  Second_Block = add_denseblock(First_Transition, num_layers,growth_rate, dropout_rate,bn_layers,middle_layers)
  num_filter+=growth_rate*num_layers
  Second_Transition = add_transition(Second_Block, growth_rate*2, dropout_rate)
  num_filter=num_filter//2
  
  Third_Block = add_denseblock(Second_Transition,num_layers, growth_rate, dropout_rate,bn_layers,middle_layers)
  num_filter+=growth_rate*num_layers
  Third_Transition = add_transition(Third_Block, growth_rate*2, dropout_rate)
  
  Last_Block = add_denseblock(Third_Transition, num_layers, growth_rate, dropout_rate,bn_layers,middle_layers)
  output = output_layer(Third_Block)
  model = Model(inputs=[input], outputs=[output])

In [0]:
# tbc=TensorBoardColabCallback(TensorBoardColab())

In [0]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_540 (Conv2D)             (None, 32, 32, 12)   324         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_540 (BatchN (None, 32, 32, 12)   48          conv2d_540[0][0]                 
__________________________________________________________________________________________________
activation_540 (Activation)     (None, 32, 32, 12)   0           batch_normalization_540[0][0]    
__________________________________________________________________________________________________
conv2d_541

In [0]:
print('====================HYPER PARAMETERS====================')

====================HYPER PARAMETERS====================


In [0]:
#load data
x_train, y_train, x_test, y_test = load_data(resize=False,append=False)#,train_augment=False
hist=[]

Before augmentation: (50000, 32, 32, 3) (50000, 10)
After augmentation: (50000, 32, 32, 3) (50000, 10)


In [0]:
#callbacks
batch_size=128
modval=8
max_lr=0.3

model_checkpointer=ModelCheckpoint(weights_dir+'large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))

use_cosine_annealing=False
lrs=LearningRateScheduler(cosine_anneal_schedule) if use_cosine_annealing else clr
callbacks = [model_checkpointer,lrs]
initial_epoch=epochs
epochs=24
epochs+=initial_epoch

h=model.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=initial_epoch,epochs=epochs, callbacks=callbacks, validation_data=(x_test,y_test))
hist.append(h)
model.save(model_file)
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 25/48
50000/50000 [==============================] - 323s 6ms/step - loss: 2.0217 - acc: 0.2341 - val_loss: 1.9416 - val_acc: 0.2877
Epoch 26/48
50000/50000 [==============================] - 300s 6ms/step - loss: 1.8842 - acc: 0.3007 - val_loss: 1.9554 - val_acc: 0.3198

Epoch 00026: val_acc improved from -inf to 0.31980, saving model to /content/gdrive/My Drive/Colab Notebooks/weights/large_weights.26-0.32.h5
Epoch 27/48
50000/50000 [==============================] - 303s 6ms/step - loss: 1.8387 - acc: 0.3230 - val_loss: 1.8963 - val_acc: 0.3272
Epoch 28/48
50000/50000 [==============================] - 303s 6ms/step - loss: 1.8030 - acc: 0.3403 - val_loss: 1.8915 - val_acc: 0.3391

Epoch 00028: val_acc improved from 0.31980 to 0.33910, saving model to /content/gdrive/My Drive/Colab Notebooks/weights/large_weights.28-0.34.h5
Epoch 29/48
50000/50000 [==============================] - 303s 6ms/step - loss: 1.7739 - acc: 0.3520 - v

KeyboardInterrupt: ignored

In [0]:
#callbacks
batch_size=100
modval=8
max_lr=0.3

model_checkpointer=ModelCheckpoint(weights_dir+'large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
clr = CyclicLR(base_lr=0.01, max_lr=0.2,step_size=4*(len(y_train)/batch_size))

use_cosine_annealing=False
lrs=LearningRateScheduler(cosine_anneal_schedule) if use_cosine_annealing else clr
callbacks = [model_checkpointer,lrs]
initial_epoch=epochs
epochs=8
epochs+=initial_epoch

h=model.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=initial_epoch,epochs=epochs, callbacks=callbacks, validation_data=(x_test,y_test))
hist.append(h)
model.save(model_file)
print("Saved model to disk")

In [0]:
#callbacks
batch_size=128
modval=8
max_lr=0.3

model_checkpointer=ModelCheckpoint(weights_dir+'large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
clr = CyclicLR(base_lr=0.01, max_lr=0.1,step_size=8*(len(y_train)/batch_size))

use_cosine_annealing=False
lrs=LearningRateScheduler(cosine_anneal_schedule) if use_cosine_annealing else clr
callbacks = [model_checkpointer,lrs]
initial_epoch=epochs
epochs=8
epochs+=initial_epoch

h=model.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=initial_epoch,epochs=epochs, callbacks=callbacks, validation_data=(x_test,y_test))
hist.append(h)
model.save(model_file)
print("Saved model to disk")

In [0]:
# x_train, y_train, x_test, y_test = load_data(resize=False,append=True)#,train_augment=False

In [0]:
#callbacks
batch_size=128
modval=8
max_lr=0.3

model_checkpointer=ModelCheckpoint(weights_dir+'large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
clr = CyclicLR(base_lr=0.001, max_lr=0.1,step_size=8*(len(y_train)/batch_size))

use_cosine_annealing=False
lrs=LearningRateScheduler(cosine_anneal_schedule) if use_cosine_annealing else clr
callbacks = [model_checkpointer,lrs]
initial_epoch=epochs
epochs=40
epochs+=initial_epoch

h=model.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=initial_epoch,epochs=epochs, callbacks=callbacks, validation_data=(x_test,y_test))
hist.append(h)
model.save(model_file)
print("Saved model to disk")

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
print('====================END OF LARGER MODEL====================')